<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/GrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

def calculate_average_growth_rate(data, start_year, end_year, column):
    start_value = data.loc[data['year'] == start_year, column].iloc[0]
    end_value = data.loc[data['year'] == end_year, column].iloc[0]
    years = end_year - start_year
    avg_growth_rate = np.exp(np.log(end_value / start_value) / years) - 1
    return avg_growth_rate

# Load data
url = 'https://dataverse.nl/api/access/datafile/354098'
pwt1001 = pd.read_stata(url)

# Filter and select relevant columns
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'country', 'rgdpna', 'rnna', 'emp', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1990) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['gdp_per_worker'] = data['rgdpna'] / data['emp']
data['capital_per_worker'] = data['rnna'] / data['emp']

# Calculate average growth rates for each country
results = []
for country in countries:
    country_data = data[data['country'] == country]
    alpha = country_data['alpha'].mean()
    gdp_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'gdp_per_worker')
    capital_deepening = calculate_average_growth_rate(country_data, 1990, 2019, 'capital_per_worker') * alpha
    tfp_growth = gdp_growth - capital_deepening
    tfp_share = tfp_growth / gdp_growth
    capital_share = capital_deepening / gdp_growth
    
    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepening': capital_deepening,
        'TFP Share': tfp_share,
        'Capital Share': capital_share
    })

# Create a DataFrame from the results
growth_accounting_table = pd.DataFrame(results)

# Calculate the average values for each column
average_values = {
    'Country': 'Average',
    'Growth Rate': growth_accounting_table['Growth Rate'].mean(),
    'TFP Growth': growth_accounting_table['TFP Growth'].mean(),
    'Capital Deepening': growth_accounting_table['Capital Deepening'].mean(),
    'TFP Share': growth_accounting_table['TFP Share'].mean(),
    'Capital Share': growth_accounting_table['Capital Share'].mean()
}

# Convert the average_values dictionary to a DataFrame
average_values_df = pd.DataFrame([average_values])

# Concatenate the average_values_df with the growth_accounting_table
growth_accounting_table = pd.concat([growth_accounting_table, average_values_df], ignore_index=True)

# Display the updated table
print(growth_accounting_table.to_string(index=False, float_format='{:.2%}'.format))

       Country  Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
     Australia        1.28%       0.87%              0.41%     68.09%         31.91%
       Austria        1.04%       0.48%              0.56%     45.77%         54.23%
       Belgium        0.95%       0.34%              0.61%     36.10%         63.90%
        Canada        0.98%       0.45%              0.52%     46.36%         53.64%
       Denmark        1.36%       0.86%              0.50%     63.29%         36.71%
       Finland        1.45%       0.79%              0.65%     54.81%         45.19%
        France        0.91%       0.46%              0.45%     50.82%         49.18%
       Germany        1.04%       0.58%              0.47%     55.44%         44.56%
        Greece        0.73%       0.18%              0.55%     24.25%         75.75%
       Iceland        1.75%       1.47%              0.28%     84.11%         15.89%
       Ireland        3.17%       1.62%              1.56%     50